## Import Libraries

In [2]:
import tensorflow as tf
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GlobalAveragePooling1D,GRU,LSTM,Bidirectional,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import zipfile
import re
import string
import nltk
import torch

## Data Downloading

In [ ]:
# zip_path = "/content/IMDB Dataset.csv.zip"
# extract_to = "data/"
# # Open the zip file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     # Extract specific files
#     zip_ref.extract("IMDB Dataset.csv", extract_to)

In [3]:
train_data = pd.read_csv("/kaggle/input/dataset-imdb/IMDB Dataset.csv")
train_df, test_df = train_test_split(train_data, test_size=0.1, random_state=42)

In [4]:
# Tải stopwords nếu chưa có
nltk.download("stopwords")

# Khai báo stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Data Preprocessing

In [5]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = " ".join([word for word in text.split() if word not in stop_words])
    text = re.sub(r"\s+", " ", text).strip()
    return text

train_df["review"] = train_df["review"].apply(preprocess_text)
test_df["review"] = test_df["review"].apply(preprocess_text)

In [6]:
# separate review and sentiment
X_train = train_df["review"].values
y_train = train_df["sentiment"].values
X_test = test_df["review"].values
y_test = test_df["sentiment"].values

print("y_train type:", type(y_train))
print("Unique values in y_train:", np.unique(y_train))

# map 1->positive, 0->negative
if y_train.dtype == 'O':
    label_mapping = {'positive': 1, 'negative': 0}
    y_train = np.array([label_mapping[label] for label in y_train], dtype=np.float32)
    y_test = np.array([label_mapping[label] for label in y_test], dtype=np.float32)
else:
    y_train = y_train.astype(np.float32)
    y_test = y_test.astype(np.float32)

MAX_WORDS = 10000
MAX_LEN = 500

EMBEDDING_DIMS = 128

# Tokenize
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

y_train type: <class 'numpy.ndarray'>
Unique values in y_train: ['negative' 'positive']


## Build Model

#### Vanilla RNN Model

In [7]:
# Vanilla RNN model
def create_rnn_model():
    model = Sequential([
        Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIMS, mask_zero=True),  # mask_zero=True
        SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=False),  # return_sequences=False -> output 2D
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#### GRU Model

In [ ]:

# from tensorflow.keras.layers import GRU, Dense, Embedding, Bidirectional, BatchNormalization, Dropout
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2

# def create_gru_model():
#     model = Sequential([
#         # Embedding Layer với mask_zero để xử lý padding
#         Embedding(
#             input_dim=MAX_WORDS,
#             output_dim=EMBEDDING_DIMS,
#             input_length=MAX_LEN,
#             mask_zero=True
#         ),

#         # Stacked Bidirectional GRU Layers
#         GRU(
#             128,
#             return_sequences=True,
#             dropout=0.3,          # Dropout cho đầu vào
#             recurrent_dropout=0.2, # Dropout cho recurrent state
#             kernel_regularizer=l2(1e-4)
#         ),
#         BatchNormalization(),

#         GRU(
#             64,
#             return_sequences=False, # Layer cuối không cần return sequences
#             dropout=0.2,
#             recurrent_dropout=0.1
#         ),
#         Dropout(0.3),

#         # Hidden Layers với regularization
#         Dense(32, activation='relu', kernel_regularizer=l2(1e-4)),
#         BatchNormalization(),
#         Dropout(0.2),

#         # Output Layer
#         Dense(1, activation='sigmoid')
#     ])

#     # optimize
#     optimizer = Adam(
#         learning_rate=1e-3,
#         clipnorm=1.0  # avoid exploding gradients
#     )

#     model.compile(
#         optimizer=Adam(learning_rate=1e-4),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

def create_gru_model():
    model = Sequential([
        Embedding(
            input_dim=MAX_WORDS,
            output_dim=EMBEDDING_DIMS,
            input_length=MAX_LEN,
            mask_zero=True
        ),

        # Dropout cho embedding layer
        Dropout(0.3),

        # GRU Layer 1
        GRU(
            128,
            return_sequences=True,
            dropout=0.3,
            recurrent_dropout=0.2,
            kernel_regularizer=l2(1e-4)
        ),
        # GRU Layer 2
        GRU(
            64,
            dropout=0.2,
            recurrent_dropout=0.1
        ),
        # Hidden Layer với regularization
        Dense(32, activation='relu', kernel_regularizer=l2(1e-4)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model



#### LSTM Model

In [11]:
# LSTM model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, BatchNormalization, Dense
from tensorflow.keras.regularizers import l2

def create_lstm_model():
    model = Sequential([
        # embedding
        Embedding(input_dim=MAX_WORDS, output_dim=64, mask_zero=True),

        # optimized bidirectional LSTM
        Bidirectional(LSTM(32, return_sequences=True)),
        BatchNormalization(),

        Bidirectional(LSTM(16)),
        Dropout(0.1),

        # dense layers with reduced regularization
        Dense(16, activation="relu"),
        BatchNormalization(),

        # Output layer
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

## Train Model

#### Train Vanilla RNN Model

In [9]:
# convert text data to sequences of integers using the tokenizer
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# padding sequence
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")


In [10]:
# train Vanilla RNN model
rnn_model = create_rnn_model()
X_train_tensor = np.array(X_train_pad, dtype=np.int32)
y_train_tensor = np.array(y_train, dtype=np.float32)
X_test_tensor = np.array(X_test_pad, dtype=np.int32)
y_test_tensor = np.array(y_test, dtype=np.float32)

# train model
history_rnn = rnn_model.fit(
    X_train_tensor, y_train_tensor,
    validation_data=(X_test_tensor, y_test_tensor),
    epochs=10, batch_size=64
)


Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 52ms/step - accuracy: 0.5147 - loss: 0.7024 - val_accuracy: 0.6222 - val_loss: 0.6558
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.6309 - loss: 0.6306 - val_accuracy: 0.7276 - val_loss: 0.5360
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.7513 - loss: 0.5041 - val_accuracy: 0.7108 - val_loss: 0.5507
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.7656 - loss: 0.4848 - val_accuracy: 0.7066 - val_loss: 0.5634
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.7752 - loss: 0.4731 - val_accuracy: 0.7874 - val_loss: 0.4935
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.8401 - loss: 0.3723 - val_accuracy: 0.7828 - val_loss: 0.4761
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.8446 - loss: 0.3675 - val_accuracy: 0.6920 - val_loss: 0.5859
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 48ms/step - accuracy: 0.7776 - loss: 0.4619 - 

#### Train GRU Model

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# padding
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")


In [ ]:
# train GRU model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
gru_model = create_gru_model()
history_gru = gru_model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test),
                            epochs=5, batch_size=128, callbacks=[early_stopping])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 817s 2s/step - accuracy: 0.5560 - loss: 0.6965 - val_accuracy: 0.8276 - val_loss: 0.4052
Epoch 2/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 867s 2s/step - accuracy: 0.8353 - loss: 0.4059 - val_accuracy: 0.8600 - val_loss: 0.3388
Epoch 3/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 810s 2s/step - accuracy: 0.8714 - loss: 0.3312 - val_accuracy: 0.8772 - val_loss: 0.3064
Epoch 4/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 862s 2s/step - accuracy: 0.8860 - loss: 0.3029 - val_accuracy: 0.8842 - val_loss: 0.2923
Epoch 5/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 864s 2s/step - accuracy: 0.8987 - loss: 0.2749 - val_accuracy: 0.8858 - val_loss: 0.2842


In [16]:
# Define tokenizer (adjust num_words if needed)
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)  # Fit tokenizer on training data

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# pad the sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')

In [17]:
# Train LSTM model
lstm_model = create_lstm_model()
history_lstm = lstm_model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test),
                              epochs=5, batch_size=64, callbacks=[early_stopping])

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.5840 - loss: 0.7170 - val_accuracy: 0.8044 - val_loss: 0.4370
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 43s 62ms/step - accuracy: 0.8422 - loss: 0.3611 - val_accuracy: 0.8642 - val_loss: 0.3137
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 43s 62ms/step - accuracy: 0.9099 - loss: 0.2302 - val_accuracy: 0.8828 - val_loss: 0.2741


### Plot results

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history_rnn.history['val_accuracy'], label="Vanilla RNN")
plt.plot(history_gru.history['val_accuracy'], label="GRU")
plt.plot(history_lstm.history['val_accuracy'], label="LSTM")
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
plt.legend()
plt.title("Model Accuracy Comparison")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy(history):
    """
    Plots training and validation accuracy over epochs.

    Parameters:
    history (dict or keras.callbacks.History):
        A dictionary or Keras History object containing 'accuracy' and 'val_accuracy'.
    """
    if hasattr(history, 'history'):  # If it's a Keras History object, extract history
        history = history.history

    epochs = range(1, len(history['accuracy']) + 1)

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, history['accuracy'], label='Training Accuracy', marker='o')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy', marker='s')

    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_accuracy(lstm_model.history)

In [ ]:
plot_accuracy(rnn_model.history)